##This netbook uses the points from depth estimation to get required side measurements


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

Getting contours

In [ ]:
print("Enter user height")
height=input()
print("Enter user image path")
img_path=input()
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
a=gray.max()
flag, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
# Find contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True) 

perimeters = [cv2.arcLength(contours[i],True) for i in range(len(contours))]
listindex=[i for i in range(15) if perimeters[i]>perimeters[0]/2]
numcards=len(listindex)

card_number = -1 
stencil = np.zeros(img.shape).astype(img.dtype)
cv2.drawContours(stencil, [contours[listindex[card_number]]], 0, (255, 255, 255), cv2.FILLED)
res = cv2.bitwise_and(img, stencil)
cnt = cv2.Canny(res, 100, 200)

Enter user height
156
Enter user image path
/content/mside.png


Get the points estimated from depth estimations defined as "points"

In [ ]:
#substitute the points here
points=[(268, 11), (256, 81), (244, 104), (244, 198), (279, 302), (244, 104), (221, 174), (279, 291), (256, 279), (256, 372), (256, 477), (268, 268), (268, 372), (256, 466), (256, 198)]

In [ ]:
userheight=int(height)-8 #considering the ankle error
calculatedheight=(points[13][1]-points[0][1])
ratio=(userheight/calculatedheight)#calculating actual height to pixcel ratio
print(ratio)

0.3252747252747253


bust side

In [ ]:
hiplimit=points[8][1]
shoulderlimit=points[5][1]
verticallimit=points[0][0]
curr_min=0
for c in contours:
    for i in c:
        if(i[0][0]>verticallimit and i[0][1]<hiplimit and i[0][1]>shoulderlimit):
            if i[0][0]>curr_min:
                curr_min=i[0][0]
                bustpoint=i
bustz=(bustpoint[0][0]-points[2][0])*ratio
print(bustz)

22.443956043956046


waist side

In [ ]:
currwaist=100000
for c in contours:
    for i in c:
        if(i[0][0]<points[0][0] and i[0][1]>bustpoint[0][1] and i[0][1]<points[8][1] and points[0][0]-i[0][0]<currwaist):
            currwaist=points[0][0]-i[0][0]
            waistpoint=i
curr_waist=0
waistextreme=[]
for c in contours:
    for j in c:
        if(j[0][0]>points[0][0] and j[0][1]==waistpoint[0][1] and j[0][0]-waistpoint[0][0]>curr_waist):
            curr_waist=j[0][0]-waistpoint[0][0]
            waistextreme=j[0][0]-waistpoint[0][0]
if(not waistextreme):
        waistz=(max(points[8][0],points[11][0])-waistpoint[0][0])*2*ratio
else:
    waistz=(waistextreme[0][0]-waistpoint[0][0])*ratio
print(waistz)

16.263736263736263


hip side

In [ ]:
refa=points[8][1]
refb=points[11][1]
currenthip=0
hippoint=[]
for c in contours:
    for i in c:
        if(i[0][0]<points[0][0] and (i[0][1]==refa or i[0][1]==refb) and points[0][0]-i[0][0]>currenthip):
            currenthip=points[0][0]-i[0][0]
            hippoint=i
if(hippoint):
    curr_hip=0
    hipextreme=[]
    for c in contours:
        for j in c:
            if(j[0][0]>points[0][0] and j[0][1]==hippoint[0][1] and j[0][0]-hippoint[0][0]>curr_hip):
                curr_hip=j[0][0]-hippoint[0][0]
                hipextreme=j
    if(not hipextreme):
            hipz=(max(points[8][0],points[11][0])-hippoint[0][0])*2*ratio
    else:
        hipz=(hipextreme[0][0]-hippoint[0][0])*ratio
    print(hipz)
else:
    hipz=bustz
    print(hipz)

22.443956043956046


Final measurements

In [ ]:
#to get the final bust measurement subsistitute bustz and bustx from size calucaltion in the formula given
bust=2*math.pi*math.sqrt(((bustx)**2)+((bustz)**2)/2)

In [ ]:
#to get the final waist measurement subsistitute waistz and waistx from size calucaltion in the formula given
waist=2*math.pi*math.sqrt(((waistx)**2)+((waistz)**2)/2)

In [ ]:
#to get the final hip measurement subsistitute hipz and hipx from size calucaltion in the formula given
hip=2*math.pi*math.sqrt(((hipx)**2)+((hipz)**2)/2)